# 인터파크 베스트셀러 크롤링

In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

# 1.데이터가져오기

In [ ]:
# 순위, 제목, 저자 , 역자(있으면 가져오고 없으면 '-'), 출판사, 쪽수, 발행일, 가격 

In [2]:
# http://book.interpark.com/display/collectlist.do?_method=BestsellerHourNew201605&bestTp=1&dispNo=028&smid1=s_menu&smid2=bestseller
site = 'https://book.interpark.com'
url = site + '/display/collectlist.do?_method=BestsellerHourNew201605&bestTp=1&dispNo=028&smid1=s_menu&smid2=bestseller'
res = requests.get(url)

In [3]:
soup = BeautifulSoup(res.text, 'html.parser')

# 원하는 데이터 찾기

In [4]:
lis = soup.select('.rankBestContentList > ol > li')
len(lis)

15

In [5]:
li = lis[1]

In [6]:
href = li.select_one(".coverImage").find('a')['href']
href

'/product/BookDisplay.do?_method=detail&sc.shopNo=0000400000&sc.prdNo=356690382&sc.saNo=003003001&bid1=Best_zone&bid2=LiveRanking&bid3=PRD&bid4=001'

In [7]:
#책제목 
title = li.select_one('.itemName').get_text().strip()
title

'메리골드 마음 세탁소'

In [8]:
#출판사 
compay = li.select_one('.company').get_text().strip()
compay

'북로망스'

In [9]:
#가격
price = li.select_one('.price > em').get_text()
price = int(price.replace(',', ''))
price

13500

In [10]:
# 1~9 순위
# li.select_one('.rankBtn_ctrl')
# li.select_one('.rankBtn_ctrl')['class'][1][-1]
rank = int(li.select_one('.rankBtn_ctrl')['class'][1][-1])
rank

2

In [11]:
# 10~15순위
element = '<div class="rankNumber digit2"> <span class="rankBtn_ctrl rkNum_M01 "></span><span class="rankBtn_ctrl rkNum_M04 "></span> </div>'
mini_soup = BeautifulSoup(element, 'html.parser')

In [12]:
mini_soup.select('.rankBtn_ctrl')

[<span class="rankBtn_ctrl rkNum_M01"></span>,
 <span class="rankBtn_ctrl rkNum_M04"></span>]

In [13]:
rank_data = li.select('.rankBtn_ctrl')
if len(li.select('.rankBtn_ctrl')) == 1:
    rank = int(li.select_one('.rankBtn_ctrl')['class'][1][-1])
else: 
    rank = rank_data[0]['class'][1][-1] + rank_data[1]['class'][1][-1]
    rank = int(rank)

# sub page

In [14]:
url = site + href
res = requests.get(url)
sub_soup = BeautifulSoup(res.text, 'html.parser')

In [15]:
sub_lis = sub_soup.select(".bInfo_txt > li")
len(sub_lis)

5

In [16]:
sub_lis

[<li>저 : <a href="#" onclick='fn_wiseLogRedirect("infor","basic_inform","writer","http://bsearch.interpark.com/dsearch/book.jsp?sch=all&amp;sc.shopNo=&amp;bookblockname=s_main&amp;booklinkname=s_main&amp;query=윤정은","_blank");return false;'>윤정은</a></li>,
 <li>출판사 : <a href="#" id="hdelvMafcEntrNm" onclick='fn_wiseLogRedirect("infor","basic_inform","publisher","http://bsearch.interpark.com/dsearch/book.jsp?sch=all&amp;query=북로망스","_blank");return false;'>북로망스</a></li>,
 <li>발행 : 2023년 03월 06일</li>,
 <li>쪽수 : 272</li>,
 <li>ISBN : 9791191891287</li>]

In [17]:
for li in sub_lis:
    print(li.get_text())

저 : 윤정은
출판사 : 북로망스
발행 : 2023년 03월 06일
쪽수 : 272
ISBN : 9791191891287


In [21]:
author = '-'
trans = '-'
for li in sub_lis:
    text = li.get_text().strip()
#     print(text)
    if '발행 :' in text:
        date = text.split(':')[1].strip()
        date = re.sub('[^0-9]','', date)
        #print(date)
    elif "쪽수 : " in text:
        page = int(text.split(':')[1].strip())
    elif "역 : " in text:
        trans = text.split(':')[1].strip()
    elif "저 : " in text:
        author = text.split(':')[1].strip()

In [22]:
print(date, page, trans, author)

20230306 272 - 윤정은


# 페이지에 있는 모든 데이터 추출(tqdm)

In [24]:
from tqdm.notebook import tqdm

In [25]:
lines = []
for li in tqdm(lis):
    href = li.select_one(".coverImage").find('a')['href']
    title = li.select_one('.itemName').get_text().strip()
    company = li.select_one('.company').get_text().strip()
    price = li.select_one('.price > em').get_text()
    price = int(price.replace(',', ''))
    rank_data = li.select('.rankBtn_ctrl')
    if len(li.select('.rankBtn_ctrl')) == 1:
        rank = int(li.select_one('.rankBtn_ctrl')['class'][1][-1])
    else: 
        rank = rank_data[0]['class'][1][-1] + rank_data[1]['class'][1][-1]
        rank = int(rank)
    url = site + href
    res = requests.get(url)
    sub_soup = BeautifulSoup(res.text, 'html.parser')
    sub_lis = sub_soup.select(".bInfo_txt > li")
    author = '-'
    trans = '-'
    for li in sub_lis:
        text = li.get_text().strip()
        if '발행 :' in text:
            date = text.split(':')[1].strip()
            date = re.sub('[^0-9]','', date)
        elif "쪽수 : " in text:
            page = int(text.split(':')[1].strip())
        elif "역 : " in text:
            trans = text.split(':')[1].strip()
        elif "저 : " in text:
            author = text.split(':')[1].strip()
    lines.append([rank, title, author, trans, company, page, date, price])

  0%|          | 0/15 [00:00<?, ?it/s]

In [26]:
df = pd.DataFrame(lines, 
    columns=['순위', '제목', '저자', '역자', '출판사', '쪽수', '발행일', '가격'])
df

,순위,제목,저자,역자,출판사,쪽수,발행일,가격
0,1,세이노의 가르침,세이노,-,데이원,0,20230302,6480
1,2,메리골드 마음 세탁소,윤정은,-,북로망스,272,20230306,13500
2,3,장하준의 경제학 레시피,장하준,-,부키,376,20230330,16200
3,4,이미 늦었다고 생각하는 당신을 위한 김미경의 마흔 수업,김미경,-,어웨이크북스,300,20230215,16200
4,4,영어책 한 권 외워봤니?,-,-,위즈덤하우스,0,20170111,12600
5,6,"괜찮아, 그 길 끝에 행복이 기다릴 거야",손미나,-,코알라컴퍼니,296,20230405,17100
6,7,한국형 가치투자,"김민국, 최준철",-,이콘,296,20230315,16200
7,7,K 배터리 레볼루션,박순혁,-,지와인,0,20230220,17100
8,7,불편한 편의점 2(단풍 에디션),김호연,-,나무옆의자,320,20220810,12600
9,10,브라질에 비가 내리면 스타벅스 주식을 사라,피터 나바로,이창식,에프엔미디어,0,20220425,16200


In [27]:
df.to_csv('230418인터파크베스트셀러.csv', index=False)